In [5]:
import pandas as pd
import plotly.express as px
import matplotlib.colors as mcolors
import ipywidgets as widgets
from IPython.display import display, clear_output

# ------------------------------------------------------------------
# DATASET AND PREPARATION
# ------------------------------------------------------------------
data = [
    # Romance
    {"country": "Andorra", "iso_alpha": "AND", "language_family": "Romance", "hover_info": "Info for Andorra"},
    {"country": "France", "iso_alpha": "FRA", "language_family": "Romance", "hover_info": "Info for France"},
    {"country": "Italy", "iso_alpha": "ITA", "language_family": "Romance", "hover_info": "Info for Italy"},
    {"country": "Portugal", "iso_alpha": "PRT", "language_family": "Romance", "hover_info": "Info for Portugal"},
    {"country": "Romania", "iso_alpha": "ROU", "language_family": "Romance", "hover_info": "Info for Romania"},
    {"country": "Moldova", "iso_alpha": "MDA", "language_family": "Romance", "hover_info": "Info for Moldova"},
    {"country": "San Marino", "iso_alpha": "SMR", "language_family": "Romance", "hover_info": "Info for San Marino"},
    {"country": "Monaco", "iso_alpha": "MCO", "language_family": "Romance", "hover_info": "Info for Monaco"},
    {"country": "Vatican City", "iso_alpha": "VAT", "language_family": "Romance", "hover_info": "Info for Vatican City"},
    {"country": "Spain", "iso_alpha": "ESP", "language_family": "Romance", "hover_info": "Info for Spain"},
    # Germanic
    {"country": "Austria", "iso_alpha": "AUT", "language_family": "Germanic", "hover_info": "Info for Austria"},
    {"country": "Germany", "iso_alpha": "DEU", "language_family": "Germanic", "hover_info": "Info for Germany"},
    {"country": "United Kingdom", "iso_alpha": "GBR", "language_family": "Germanic", "hover_info": "Info for United Kingdom"},
    {"country": "Denmark", "iso_alpha": "DNK", "language_family": "Germanic", "hover_info": "Info for Denmark"},
    {"country": "Netherlands", "iso_alpha": "NLD", "language_family": "Germanic", "hover_info": "Info for Netherlands"},
    {"country": "Norway", "iso_alpha": "NOR", "language_family": "Germanic", "hover_info": "Info for Norway"},
    {"country": "Sweden", "iso_alpha": "SWE", "language_family": "Germanic", "hover_info": "Info for Sweden"},
    {"country": "Iceland", "iso_alpha": "ISL", "language_family": "Germanic", "hover_info": "Info for Iceland"},
    {"country": "Ireland", "iso_alpha": "IRL", "language_family": "Germanic", "hover_info": "Info for Ireland"},
    {"country": "Liechtenstein", "iso_alpha": "LIE", "language_family": "Germanic", "hover_info": "Info for Liechtenstein"},
    # Slavic
    {"country": "Belarus", "iso_alpha": "BLR", "language_family": "Slavic", "hover_info": "Info for Belarus"},
    {"country": "Bosnia and Herzegovina", "iso_alpha": "BIH", "language_family": "Slavic", "hover_info": "Info for Bosnia and Herzegovina"},
    {"country": "Bulgaria", "iso_alpha": "BGR", "language_family": "Slavic", "hover_info": "Info for Bulgaria"},
    {"country": "Croatia", "iso_alpha": "HRV", "language_family": "Slavic", "hover_info": "Info for Croatia"},
    {"country": "Czech Republic", "iso_alpha": "CZE", "language_family": "Slavic", "hover_info": "Info for Czech Republic"},
    {"country": "North Macedonia", "iso_alpha": "MKD", "language_family": "Slavic", "hover_info": "Info for North Macedonia"},
    {"country": "Montenegro", "iso_alpha": "MNE", "language_family": "Slavic", "hover_info": "Info for Montenegro"},
    {"country": "Poland", "iso_alpha": "POL", "language_family": "Slavic", "hover_info": "Info for Poland"},
    {"country": "Russia", "iso_alpha": "RUS", "language_family": "Slavic", "hover_info": "Info for Russia"},
    {"country": "Serbia", "iso_alpha": "SRB", "language_family": "Slavic", "hover_info": "Info for Serbia"},
    {"country": "Slovakia", "iso_alpha": "SVK", "language_family": "Slavic", "hover_info": "Info for Slovakia"},
    {"country": "Slovenia", "iso_alpha": "SVN", "language_family": "Slavic", "hover_info": "Info for Slovenia"},
    {"country": "Ukraine", "iso_alpha": "UKR", "language_family": "Slavic", "hover_info": "Info for Ukraine"},
    # Others / Multilingual
    {"country": "Luxembourg", "iso_alpha": "LUX", "language_family": "Multilingual", "language_families": ["Germanic", "Romance"], "hover_info": "Info for Luxembourg"},
    {"country": "Switzerland", "iso_alpha": "CHE", "language_family": "Multilingual", "language_families": ["Germanic", "Romance"], "hover_info": "Info for Switzerland"},
    {"country": "Belgium", "iso_alpha": "BEL", "language_family": "Multilingual", "language_families": ["Germanic", "Romance"], "hover_info": "Belgium has multiple languages: Dutch, French, and German."},
    {"country": "Cyprus", "iso_alpha": "CYP", "language_family": "Others", "hover_info": "Info for Cyprus"},
    {"country": "Estonia", "iso_alpha": "EST", "language_family": "Others", "hover_info": "Info for Estonia"},
    {"country": "Finland", "iso_alpha": "FIN", "language_family": "Others", "hover_info": "Info for Finland"},
    {"country": "Greece", "iso_alpha": "GRC", "language_family": "Others", "hover_info": "Info for Greece"},
    {"country": "Hungary", "iso_alpha": "HUN", "language_family": "Others", "hover_info": "Info for Hungary"},
    {"country": "Kosovo", "iso_alpha": "XKX", "language_family": "Others", "hover_info": "Info for Kosovo"},
    {"country": "Latvia", "iso_alpha": "LVA", "language_family": "Others", "hover_info": "Info for Latvia"},
    {"country": "Lithuania", "iso_alpha": "LTU", "language_family": "Others", "hover_info": "Info for Lithuania"},
    {"country": "Malta", "iso_alpha": "MLT", "language_family": "Others", "hover_info": "Info for Malta"},
    {"country": "Turkey", "iso_alpha": "TUR", "language_family": "Others", "hover_info": "Info for Turkey"}
]
df = pd.DataFrame(data)

# Ensure every row has a 'language_families' column as a list.
def ensure_language_families(row):
    if "language_families" in row and isinstance(row["language_families"], list):
        return row["language_families"]
    else:
        return [row["language_family"]]
df["language_families"] = df.apply(ensure_language_families, axis=1)

# ------------------------------------------------------------------
# FILTER FUNCTION WITH UPDATED EXCEPTION LOGIC
# ------------------------------------------------------------------
def filter_dataframe(df, exclude_countries, exclude_language_families, exceptions):
    df_filtered = df.copy()
    # First, exclude by country.
    if exclude_countries:
        df_filtered = df_filtered[~df_filtered["country"].isin(exclude_countries)]
    
    # Then apply language family exclusions.
    if exclude_language_families:
        if exceptions is None:
            exceptions = {}
        def should_include(row):
            # Always include multilingual countries.
            if row["language_family"] == "Multilingual":
                return True
            # Check which language families in the row are in the excluded set.
            excluded = [fam for fam in row["language_families"] if fam in exclude_language_families]
            if not excluded:
                return True
            # If any of the excluded families has this country as an exception, include it.
            for fam in excluded:
                if row["country"] in exceptions.get(fam, []):
                    return True
            return False
        df_filtered = df_filtered[df_filtered.apply(should_include, axis=1)]
    return df_filtered

# ------------------------------------------------------------------
# COLOR MAPPING AND BLENDING FUNCTION
# ------------------------------------------------------------------
base_colors = {
    "Romance": "darkred",
    "Germanic": "yellow",
    "Slavic": "darkgreen",
    "Multilingual": "#c58000",
    "Others": "gray"
}
color_map = base_colors

def blend_colors(families):
    colors = [color_map.get(fam, "gray") for fam in families]
    rgb_list = [mcolors.to_rgb(c) for c in colors]
    avg_rgb = tuple(sum(vals) / len(vals) for vals in zip(*rgb_list))
    return mcolors.to_hex(avg_rgb)

# ------------------------------------------------------------------
# MAP UPDATE FUNCTION
# ------------------------------------------------------------------
def update_map(exclude_countries, exclude_language_families, exceptions):
    df_new_filtered = filter_dataframe(df, exclude_countries, exclude_language_families, exceptions)
    df_new_filtered["blended_color"] = df_new_filtered["language_families"].apply(blend_colors)
    unique_colors = df_new_filtered["blended_color"].unique()
    new_blended_color_map = {color: color for color in unique_colors}
    
    new_fig = px.choropleth(
        df_new_filtered,
        locations="iso_alpha",
        color="blended_color",
        hover_name="country",
        hover_data=["hover_info"],
        color_discrete_map=new_blended_color_map,
        scope="europe"
    )
    
    # Overlay country names.
    for _, row in df_new_filtered.iterrows():
        new_fig.add_scattergeo(
            locations=[row["iso_alpha"]],
            text=[row["country"]],
            mode="text",
            showlegend=False
        )
    
    new_fig.update_layout(
        title_text="Linguistic Map of Europe by Language Families",
        geo=dict(landcolor="lightgray", showcountries=True, countrycolor="white"),
        width=1200,
        height=800
    )
    new_fig.update_traces(showlegend=False)
    new_fig.show()

# ------------------------------------------------------------------
# HELPER FUNCTION TO COLLECT SELECTED OPTIONS FROM CHECKBOXES
# ------------------------------------------------------------------
def get_selected_from_checkboxes(checkbox_dict):
    return [key for key, widget in checkbox_dict.items() if widget.value]

# ------------------------------------------------------------------
# BUILD THE EXCLUSION UI WITH CHECKBOXES
# ------------------------------------------------------------------
# Exclusion for Countries.
country_checkboxes = {country: widgets.Checkbox(value=False, description=country) 
                      for country in sorted(df["country"].unique())}
countries_box = widgets.VBox(list(country_checkboxes.values()))

# Exclusion for Language Families.
language_family_checkboxes = {lf: widgets.Checkbox(value=False, description=lf) 
                              for lf in sorted(df["language_family"].unique())}
language_family_box = widgets.VBox(list(language_family_checkboxes.values()))

accordion_exclusions = widgets.Accordion(children=[countries_box, language_family_box])
accordion_exclusions.set_title(0, "Exclude Countries")
accordion_exclusions.set_title(1, "Exclude Language Families")

# ------------------------------------------------------------------
# BUILD THE EXCEPTIONS UI WITH CHECKBOXES (Inside an Accordion)
# ------------------------------------------------------------------
exceptions_checkboxes = {}
for lf in sorted(df["language_family"].unique()):
    countries_for_lf = sorted(df[df["language_family"] == lf]["country"].unique())
    exceptions_checkboxes[lf] = {country: widgets.Checkbox(value=False, description=country) 
                                 for country in countries_for_lf}

exceptions_vboxes = []
for lf, cb_dict in exceptions_checkboxes.items():
    vbox = widgets.VBox(list(cb_dict.values()))
    exceptions_vboxes.append(vbox)

exceptions_accordion = widgets.Accordion(children=exceptions_vboxes)
for i, lf in enumerate(sorted(df["language_family"].unique())):
    exceptions_accordion.set_title(i, f"Exceptions for {lf}")

# ------------------------------------------------------------------
# SET UP THE MAP OUTPUT AREA
# ------------------------------------------------------------------
map_out = widgets.Output()

# ------------------------------------------------------------------
# UPDATE BUTTON AND WRAPPER FUNCTION
# ------------------------------------------------------------------
update_button = widgets.Button(description="Update Map")

def on_update_button_clicked(b):
    # Gather selected exclusions.
    selected_countries = get_selected_from_checkboxes(country_checkboxes)
    selected_language_families = get_selected_from_checkboxes(language_family_checkboxes)
    # Gather exceptions.
    exceptions = {}
    for lf, cb_dict in exceptions_checkboxes.items():
        exceptions[lf] = get_selected_from_checkboxes(cb_dict)
    with map_out:
        clear_output(wait=True)
        update_map(selected_countries, selected_language_families, exceptions)

update_button.on_click(on_update_button_clicked)

# ------------------------------------------------------------------
# ASSEMBLE AND DISPLAY THE UI
# ------------------------------------------------------------------
ui = widgets.VBox([accordion_exclusions, exceptions_accordion, update_button, map_out])
display(ui)
